# Beta-series modelling - Extracting LSA
Based on: https://nilearn.github.io/dev/auto_examples/07_advanced/plot_beta_series.html
Adapted by: Maria Clara Laport and Tiago Bortolini

## Importing packages

In [ ]:
import numpy as np
import pandas as pd
import os, fnmatch, glob
from nilearn import image
from nilearn.glm.first_level import FirstLevelModel
from nilearn.glm.first_level import make_first_level_design_matrix
from nilearn.plotting import plot_design_matrix
from nilearn.reporting import make_glm_report
from nilearn.image import mean_img, math_img
from nilearn.masking import apply_mask
import nibabel as nb

# 1- Setting Model Parameters
Setting the fixed parameters that will be used in the three types of modelling.

In [ ]:
# Set fixed (GLM) parameters

# Fmriprep space
space = 'MNI152NLin2009cAsym'

hrf_model='spm'
t_r = 1.0  

slice_time_ref=0.5

drift_model = 'cosine' # Specifies the desired drift model. Default=’cosine

high_pass = 0.0078125 # High-pass frequency in case of a cosine model (in Hz). Default=0.01.
drift_order = 1 #Order of the drift model (in case it is polynomial). Default=1.

smoothing_fwhm= 6
noise_model='ar1'

# Confounds to use in GLM
confounds2use = ['trans_x', 'trans_y', 'trans_z', 'rot_x', 'rot_y', 'rot_z']
motion_names = confounds2use

# 2- Loading all files needed and organizing them
Everything in these following chunks will be used for all types of modelling. If any modification is needed for a specific type of modelling, it will be done later.

## 2.1- Events data - creating dictionary of events
After setting the directories:
- First, we will upload all csv files with all the events information. We have one for each run of each subject. 
- Then, we will create a dictionary that organizes all these events information. These informations came from the previous part of the code (where we organized all the behavioral csv files), but putting them into a dictionary make everything easier to access in loops later, as we can access them sorting by subject and run.

In [ ]:
## Defining directories
data_dir = '/your_path/BIDS/'
PRE_PROC_dir = '/your_path/fmriprep'

## Events data
# Get CSV files list from a folder
behavioral_files = [y for x in os.walk(data_dir)
                    for y in glob.glob(os.path.join(x[0], 'sub-*_run-0*_events.tsv'))]

behavioral_files.sort()


# Create dictionary of events per subject and run

events_dict = {}

for file in behavioral_files:
     #Open events df with relevant columns for glm
    df_new = pd.read_csv(file, sep='\t', usecols=['onset','trial','duration','trial_type', 'subj', 'reward'])
    # Rename reward column to clip
    df_new = df_new.rename(columns={"reward": "clip"})
    #slipt and keep only clip number
    df_new["clip"] = df_new['clip'].str.split('/').str[1]
    # add to dictionary key with subj and run
    events_dict[(file.split('/')[6]),(file.split('_')[-2])] = df_new
    

# 3- Creating dictionaries - Functional, mask and confounds files
No mistery here, just uploading the files and creating simple dictionaries.

In [ ]:
## Get functional files list from a folder
func_files = [y for x in os.walk(PRE_PROC_dir)
                    for y in glob.glob(os.path.join(x[0], 'sub-*_task-amid_run-0*_desc-preproc_bold.nii.gz'))]
func_files.sort()

# create func dictionary
func_files_dict = {}

for file in func_files:
    # add to dictionary key with subj and run
    func_files_dict[(file.split('/')[7]),(file.split('_')[-4])] = file

    
    
## Get mask files list for GLM
mask_files = [y for x in os.walk(PRE_PROC_dir)
                    for y in glob.glob(os.path.join(x[0], 'sub-*_run-0*brain_mask.nii.gz'))]
mask_files.sort()

# create mask dictionary
mask_files_dict = {}
for file in mask_files:
    # add to dictionary key with subj and run
    mask_files_dict[(file.split('/')[7]),(file.split('_')[-4])] = file
    

    
## Get confounds files
confounds_files = [y for x in os.walk(PRE_PROC_dir)
                    for y in glob.glob(os.path.join(x[0], '*confounds_regressors.tsv'))]
confounds_files.sort()

# create dictionary for confounds per subject and run
confounds_dict = {}

for file in confounds_files:
    # Open confounds df with relevant columns for glm
    df_new = pd.read_csv(file, sep='\t', usecols= confounds2use)
    # add to dictionary key with subj and run
    confounds_dict[(file.split('/')[7]),(file.split('_')[-3])] = df_new

# 4- Performing LSA Extraction
- Creating LSA specific folders;
- Transforming the original dataframe to the format expected to perform LSA;
- Creating design matrices;
- Fitting the model and saving results (betamaps).

In [ ]:
outpath_lsa = '/your_path/GLM_LSA/'

## Creating LSA folders
for events in events_dict:
    #if events == ('sub-XXXX'): # SKIP PROBLEMATIC SUBJECTS
    #    continue    # continue here
    # Create directories for each subjects if doesn't exist
    if not os.path.exists('%s_LSA' % (events[0])):
        outdir = outpath_lsa+'%s_LSA' % events[0];
        if not os.path.exists(outdir):
            os.makedirs(outdir);
    outdir = outpath_lsa+'%s_LSA' % events[0]
    
    ## Transforming the DataFrame for LSA
    lsa_events_df = events_dict[events].copy()
    conditions = lsa_events_df['trial_type'].unique()
    condition_counter = {c: 0 for c in conditions}
    
    for i_trial, trial in lsa_events_df.iterrows():
        trial_condition = trial['trial_type']
        condition_counter[trial_condition] += 1
        run='run_'+events[1][-2:]
        # We use a unique delimiter here (``_``) that shouldn't be in the
        # original condition names
        trial_name = f'{run}_{trial_condition}_{condition_counter[trial_condition]:02d}'
        lsa_events_df.loc[i_trial, 'trial_type'] = trial_name
        # save events csv for control
        lsa_events_df.to_csv(outdir+"/"+events[0]+"_"+'run_'+events[1][-2:]+'_LSA_events.csv')
    
    

    ## Creating design matrices
    if not os.path.exists(outdir+"/"+events[0]+"_"+'run_'+events[1][-2:]+'_designmat.png'):
        func_img = nb.load(func_files_dict[events])
        n_scans = func_img.shape[-1]  # of volumes/scans per session
        frame_times = np.arange(n_scans) * t_r  # here are the corresponding frame times
        design_matrix = make_first_level_design_matrix(frame_times=frame_times,
                                                         events=lsa_events_df, 
                                                         drift_model=drift_model, 
                                                         high_pass=high_pass,
                                                         add_regs=confounds_dict[events], 
                                                         add_reg_names=motion_names,
                                                         hrf_model=hrf_model)
        # Save design matrices for control
        plot_design_matrix(design_matrix, output_file = outdir+"/"+events[0]+"_"+'run_'+events[1][-2:]+'_designmat.png')

        
        ## fitting model        
        lsa_glm = FirstLevelModel(mask_img=mask_files_dict[events],
                                  t_r= t_r,
                                  slice_time_ref=slice_time_ref,
                                  hrf_model=hrf_model,
                                  drift_model=drift_model,
                                  high_pass=high_pass,
                                standardize=True,
                              smoothing_fwhm=smoothing_fwhm,
                              noise_model=noise_model,
                              #n_jobs=1,    
                              minimize_memory=True,
                              verbose=True,
                              memory_level = 0)

        lsa_glm.fit(run_imgs = func_files_dict[events], design_matrices = design_matrix)
        
        lsa_beta_maps = {cond: [] for cond in events_dict[events]['trial_type'].unique()}
        trialwise_conditions = lsa_events_df['trial_type'].unique()

        for condition in trialwise_conditions:
            beta_map = lsa_glm.compute_contrast(condition, output_type='effect_size')
            beta_map.to_filename(outdir+"/"+events[0]+"_"+'run_'+events[1][-2:]+"_"+condition[+7:]+'_beta.nii.gz')
            #compute_contrast method returns the “unmasked” results. Mask the results and save
            #beta_map_masked = math_img("img1 * img2", img1=beta_map, img2=mask_files_dict[events])
            #beta_map_masked.to_filename(outdir+"/"+events[0]+"_"+'run_'+events[1][-2:]+"_"+condition[+7:]+'_beta.nii.gz')
            # Drop the trial number from the condition name to get the original name
            condition_name = condition.split('_')[2]
            lsa_beta_maps[condition_name].append(beta_map)

        # We can concatenate the lists of 3D maps into a single 4D beta series for
        # each condition, if we want
        lsa_beta_maps = {name: image.concat_imgs(maps) for name, maps in lsa_beta_maps.items()}

